## Download and Import packages

In [98]:
# NOTE: you CAN change this cell
# Add more to your needs
# you must place ALL pip install here
%pip install editdistance


[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [99]:
# NOTE: you CAN change this cell
# import your library here
import time
import random as rd
import re
import json

In [100]:
# NOTE: you CAN change this cell
# If you want to use your own database, download it here
# !gdown ...

## Preprocess Data

In [101]:
# province_path = 'province.txt'
# district_path = 'district.txt'
# ward_path = 'ward.txt'

In [102]:
# with open(province_path, mode = 'r', encoding='utf8') as f:
#   province = f.readlines()
#   province = list(map(lambda x: x.strip(),province))
# with open(district_path, mode = 'r', encoding='utf8') as f:
#   district = f.readlines()
#   district = list(map(lambda x: x.strip(),province))
# with open(ward_path, mode = 'r', encoding='utf8') as f:
#   ward = f.readlines()
#   ward = list(map(lambda x: x.strip(),province))

In [103]:
def read_txt(file_path):
    with open(file_path, mode = 'r', encoding='utf8') as f:
        data = f.readlines()
        data = list(map(lambda x: x.strip(), data))
    return data

def read_data(province_path, district_path, ward_path):
    province_list = read_txt(province_path)
    district_list = read_txt(district_path)
    ward_list = read_txt(ward_path)
    return province_list, district_list, ward_list

In [104]:
class Utils:

  @staticmethod
  def to_lower_case_non_accent_vietnamese(s: str) -> str: 
    s = s.lower()
    s = re.sub(r'à|á|ạ|ả|ã|â|ầ|ấ|ậ|ẩ|ẫ|ă|ằ|ắ|ặ|ẳ|ẵ', 'a', s)
    s = re.sub(r'è|é|ẹ|ẻ|ẽ|ê|ề|ế|ệ|ể|ễ', 'e', s)
    s = re.sub(r'ì|í|ị|ỉ|ĩ', 'i', s)
    s = re.sub(r'ò|ó|ọ|ỏ|õ|ô|ồ|ố|ộ|ổ|ỗ|ơ|ờ|ớ|ợ|ở|ỡ', 'o', s)
    s = re.sub(r'ù|ú|ụ|ủ|ũ|ư|ừ|ứ|ự|ử|ữ', 'u', s)
    s = re.sub(r'ỳ|ý|ỵ|ỷ|ỹ', 'y', s)
    s = re.sub(r'đ', 'd', s)
    # clean accent
    s = re.sub(r'\u0300|\u0301|\u0303|\u0309|\u0323', '', s)  # Huyền sắc hỏi ngã nặng
    s = re.sub(r'\u02C6|\u0306|\u031B', '', s)  # Â, Ê, Ă, Ơ, Ư
    return s

  @staticmethod
  def to_non_accent_vietnamese(s: str) -> str:
    s = re.sub(r'A|Á|À|Ã|Ạ|Â|Ấ|Ầ|Ẫ|Ậ|Ă|Ắ|Ằ|Ẵ|Ặ', 'A', s)
    s = re.sub(r'à|á|ạ|ả|ã|â|ầ|ấ|ậ|ẩ|ẫ|ă|ằ|ắ|ặ|ẳ|ẵ', 'a', s)
    s = re.sub(r'E|É|È|Ẽ|Ẹ|Ê|Ế|Ề|Ễ|Ệ', 'E', s)
    s = re.sub(r'è|é|ẹ|ẻ|ẽ|ê|ề|ế|ệ|ể|ễ', 'e', s)
    s = re.sub(r'I|Í|Ì|Ĩ|Ị', 'I', s)
    s = re.sub(r'ì|í|ị|ỉ|ĩ', 'i', s)
    s = re.sub(r'O|Ó|Ò|Õ|Ọ|Ô|Ố|Ồ|Ỗ|Ộ|Ơ|Ớ|Ờ|Ỡ|Ợ', 'O', s)
    s = re.sub(r'ò|ó|ọ|ỏ|õ|ô|ồ|ố|ộ|ổ|ỗ|ơ|ờ|ớ|ợ|ở|ỡ', 'o', s)
    s = re.sub(r'U|Ú|Ù|Ũ|Ụ|Ư|Ứ|Ừ|Ữ|Ự', 'U', s)
    s = re.sub(r'ù|ú|ụ|ủ|ũ|ư|ừ|ứ|ự|ử|ữ', 'u', s)
    s = re.sub(r'Y|Ý|Ỳ|Ỹ|Ỵ', 'Y', s)
    s = re.sub(r'ỳ|ý|ỵ|ỷ|ỹ', 'y', s)
    s = re.sub(r'Đ', 'D', s)
    s = re.sub(r'đ', 'd', s)
    # clean accent
    s = re.sub(r'\u0300|\u0301|\u0303|\u0309|\u0323', '', s)  # Huyền sắc hỏi ngã nặng
    s = re.sub(r'\u02C6|\u0306|\u031B', '', s)  # Â, Ê, Ă, Ơ, Ư
    return s

  def tokenize_to_term(s: str) -> str:
    tokens = re.split("[^a-zA-Z0-9]",s)
    return tokens

  def split_by_vowel(s: str) -> str:
    tokens = re.split("[ueoai]",s)
    return tokens
  
  def preprocess_terms(terms: list):
    terms = [x for x in terms if x != '']
    return terms

In [105]:
class MappingDict:
    '''
    Mapping from lowercase, w/o space to original string
    '''
    def __init__(self, province_list, district_list, ward_list):
        self.province_list_lower_wo_space = [Utils.to_lower_case_non_accent_vietnamese(x).replace(' ', '') for x in province_list]
        self.district_list_lower_wo_space = [Utils.to_lower_case_non_accent_vietnamese(x).replace(' ', '') for x in district_list]
        self.ward_list_lower_wo_space = [Utils.to_lower_case_non_accent_vietnamese(x).replace(' ', '') for x in ward_list]

        self.province_dict = dict(zip(self.province_list_lower_wo_space, province_list))
        self.district_dict = dict(zip(self.district_list_lower_wo_space, district_list))
        self.ward_dict = dict(zip(self.ward_list_lower_wo_space, ward_list))

        # add null string
        self.province_dict[''] = ''
        self.district_dict[''] = ''
        self.ward_dict[''] = ''

    def get_original_province(self, str_lower: str) -> str:
        return self.province_dict[str_lower]
    
    def get_original_district(self, str_lower: str) -> str:
        return self.district_dict[str_lower]
    
    def get_original_ward(self, str_lower: str) -> str:
        return self.ward_dict[str_lower]

In [106]:
class DistrictToProvince:
    def __init__(self, diction):
        self.diction = diction
        self.district_to_province = {}

        for province in self.diction:
            province_value = Utils.to_lower_case_non_accent_vietnamese(province["Name"]).replace(' ', '')
            for district in province["District"]:
                district_key = Utils.to_lower_case_non_accent_vietnamese(district["Name"]).replace(' ', '')
                self.district_to_province.setdefault(district_key, []).append(province_value)

    def get_province(self, district: str) -> str:
        '''
        Assume district is lowercase, w/o space
        '''
        return self.district_to_province[district]

In [107]:
class WardToProvince:
    def __init__(self, diction):
        self.diction = diction
        self.ward_to_province = {}

        for province in self.diction:
            province_value = Utils.to_lower_case_non_accent_vietnamese(province["Name"]).replace(' ', '')
            for district in province["District"]:
                for ward in district["Ward"]:
                    ward_key = Utils.to_lower_case_non_accent_vietnamese(ward["Name"]).replace(' ', '')
                    self.ward_to_province.setdefault(ward_key, []).append(province_value)

    def get_province(self, ward: str):
        return self.ward_to_province[ward]

In [108]:
class WardToDistrict:
    def __init__(self, diction):
        self.diction = diction
        self.ward_to_district = {}

        for province in self.diction:
            for district in province["District"]:
                district_value = Utils.to_lower_case_non_accent_vietnamese(district["Name"]).replace(' ', '')
                for ward in district["Ward"]:
                    ward_key = Utils.to_lower_case_non_accent_vietnamese(ward["Name"]).replace(' ', '')
                    self.ward_to_district.setdefault(ward_key, []).append(district_value)
    
    def get_district(self, ward: str) -> str:
        return self.ward_to_district[ward]

In [109]:
# test
# json_path="data/simplified_json_generated_data_vn_units_minified.json"
# dict = json.load(open(json_path, 'r', encoding='utf-8'))
# ward_to_district = WardToDistrict(dict)
# ward_to_district.get_district("tananhoi")

## Tree Traverse

### Build Tree

In [110]:
class TrieNode:
    # Info saved at the end of each word
    #    - is_end: bool
    #    - related province / district / ward
    def __init__(self):
        self.children = {}
        self.is_end = False

    def insert(self, word):
        node = self
        for c in word:
            if c not in node.children:
                node.children[c] = TrieNode()
            node = node.children[c]
        node.is_end = True

    def search(self, word):
        node = self
        for c in word:
            if c not in node.children:
                return False
            node = node.children[c]
        return node.is_end
    
    def dfs (self, node, word_list, word):
        for child in node.children:
            if node.children[child].is_end:
                word_list.append(word + child)
            self.dfs(node.children[child], word_list, word + child)
    
    def startsWith(self, prefix):
        node = self
        for c in prefix:
            if c not in node.children:
                return False
            node = node.children[c]
        word_list = []
        if node.is_end:
            word_list.append(prefix)
        self.dfs(node, word_list, prefix)
        return word_list

    def delete(self, word):     # return False if word not found, True if word deleted
        node = self
        for c in word:
            if c not in node.children:
                return False
            node = node.children[c]
        node.is_end = False
        return True

### Text Tree

In [111]:
# random words
def test_trie():
    root = TrieNode()
    root.insert("apple")
    print(root.search("apple"))   # return True
    print(root.search("app"))     # return False
    print(root.startsWith("app")) # return True
    root.insert("applepine")
    print(root.search("app"))     # return True
    print(root.startsWith("appl"))
    root.insert("app")
    print(root.startsWith("app"))
test_trie()

True
False
['apple']
False
['apple', 'applepine']
['app', 'apple', 'applepine']


In [112]:
def insert_to_trie(list):
    root = TrieNode()
    for name in list:
        root.insert(name)
    return root

In [113]:
def insert_to_tries(province_list, district_list, ward_list):
    '''
        - Input : province_list, district_list, ward_list
        - Output: dictionary of tries
        - Can insert more tries (abbrev, number, etc.)
    '''
    tries = {}
    tries['province'] = insert_to_trie(province_list)  # root of province trie
    tries['district'] = insert_to_trie(district_list)
    tries['ward'] = insert_to_trie(ward_list)
    return tries

In [114]:
province_list, district_list, ward_list = read_data('list_province.txt', 'list_district.txt', 'list_ward.txt')
tries = insert_to_tries(province_list, district_list, ward_list)

In [115]:
tries['province'].startsWith('H')

['Hà Nam',
 'Hà Nội',
 'Hà Tĩnh',
 'Hà Giang',
 'Hậu Giang',
 'Hải Phòng',
 'Hải Dương',
 'Hưng Yên',
 'Hồ Chí Minh']

## Post-Process Data

In [116]:
'''
What to do with a list of places?
'''

# 1. naive approach: pick place with longest name
def find_longest_str (strs):
    str_lens = [len(s) for s in strs]
    return strs[str_lens.index(max(str_lens))]

# 2. voting
#   - 3 lists: province, district, ward
#   - for the ward list, if its district/province is in the distric/province list, then +1 point for both the ward and the district/province
#   - for the district list, if its province is in the province list, then +1 point for both the district and the province
json_path="data/simplified_json_generated_data_vn_units_minified.json"
diction = json.load(open(json_path, 'r', encoding='utf-8'))

district_to_province = DistrictToProvince(diction)
ward_to_district = WardToDistrict(diction)
ward_to_province = WardToProvince(diction)

def pick_str_by_voting (province_list, district_list, ward_list):

    if '2' in district_list:
        district_list.remove('2')

    province_scores = [0] * len(province_list)
    district_scores = [0] * len(district_list)
    ward_scores = [0] * len(ward_list)
    
    if district_list != [] and province_list != []:
        for district_idx in range(len(district_list)):
            corr_provinces = district_to_province.get_province(district_list[district_idx])    # list of provinces that have this distric
            intersection = set(province_list) & set(corr_provinces)
            # print(f"district to province: {intersection}")
            if intersection != set():
                # plus 1 point for both district and province
                district_scores[district_idx] += 1
                province_scores[province_list.index(intersection.pop())] += 1

    if ward_list != [] and district_list != []:
        for ward_idx in range(len(ward_list)):
            corr_districts = ward_to_district.get_district(ward_list[ward_idx])    # list of districts that have this ward
            intersection = set(district_list) & set(corr_districts)
            # print(f"ward to district: {intersection}")
            if intersection != set():
                # plus 1 point for both ward and district
                ward_scores[ward_idx] += 1
                district_scores[district_list.index(intersection.pop())] += 1

    if ward_list != [] and province_list != []:
        for ward_idx in range(len(ward_list)):
            corr_provinces = ward_to_province.get_province(ward_list[ward_idx])    # list of provinces that have this ward
            intersection = set(province_list) & set(corr_provinces)
            # print(f"ward to province: {intersection}")
            if intersection != set():
                # plus 1 point for both ward and province
                ward_scores[ward_idx] += 1
                province_scores[province_list.index(intersection.pop())] += 1

    # print(province_list, district_list, ward_list)
    # print(province_scores, district_scores, ward_scores)

    if province_list == []: province = ''
    else:
        province = province_list[province_scores.index(max(province_scores))]

    if district_list == []: district = ''
    else: 
        district = district_list[district_scores.index(max(district_scores))]

    if ward_list == []: ward = ''
    else:
        ward = ward_list[ward_scores.index(max(ward_scores))]

    return province, district, ward

['hochiminh']


## Cách này để anh Hiểu làm tiếp :3

In [117]:
incorrect_dict = {}
alphabet_and_numbers = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"

for p in province_list:
  test_province = Utils.to_non_accent_vietnamese(p).lower()
  incorrect_dict[test_province] = p
  for i in range(10000):
    s = ""
    for idx in range(len(test_province)-1,-1,-1):
      if test_province[idx] == ' ': continue
      if rd.uniform(0,1) < 0.8:
        s = test_province[idx]+s
      else:
        if rd.uniform(0,1) < 1/3:
          char = rd.choice(alphabet_and_numbers)
          s = char + s
        elif rd.uniform(0,1) < 0.5:
          continue
        else:
          char = rd.choice(alphabet_and_numbers)
          s = test_province[idx] + char + s
    incorrect_dict[s] = p

In [118]:
len(incorrect_dict)

290122

In [119]:
order_dict = {}
index = []
for k in incorrect_dict.keys():
  key = sum(map(ord, k))
  if key not in order_dict.keys():
    order_dict[key] = []
    order_dict[key].append(k)


In [120]:
incorrect_dict['son la']

'Sơn La'

## Solution

In [121]:
# NOTE: you MUST change this cell
# New methods / functions must be written under class Solution.

count = 0

class Solution:
    def __init__(self):
        # list provice, district, ward for private test, do not change for any reason

        self.province_path = 'list_province.txt'
        self.district_path = 'list_district.txt'
        self.ward_path = 'list_ward.txt'


        # write your preprocess here, add more method if needed
        province_list = read_txt(self.province_path)
        district_list = read_txt(self.district_path)
        ward_list = read_txt(self.ward_path)

        # make tries
        province_list_lower = ["".join(Utils.to_lower_case_non_accent_vietnamese(province).split(" ")) for province in province_list]
        district_list_lower = ["".join(Utils.to_lower_case_non_accent_vietnamese(district).split(" ")) for district in district_list]
        ward_list_lower = ["".join(Utils.to_lower_case_non_accent_vietnamese(ward).split(" ")) for ward in ward_list]

        self.tries_lower = insert_to_tries(province_list_lower, district_list_lower, ward_list_lower)
        # print(self.tries_lower['province'].startsWith('b'))
        # print(self.tries_lower['district'].startsWith('h'))

        # make mapping dict
        self.lower_to_org_dict = MappingDict(province_list, district_list, ward_list)
        

    def process(self, s: str):
        '''Write your process string here
        '''

        # split string to words
        normalized_str = Utils.to_lower_case_non_accent_vietnamese(s)
        terms = Utils.tokenize_to_term(normalized_str)
        terms = Utils.preprocess_terms(terms)
        # print(terms)

        len_terms = len(terms)

        province, district, ward = "", "", ""

        global count
        
        # exit()
        # if (count == 5):
        #     assert(False)
        # count += 1

        provinces = []
        districts = []
        wards = []

        for term_idx in range(len_terms - 1):
            province_flag = district_flag = ward_flag = True
            for str_len in range(1, 5):
                if term_idx + str_len > len_terms:
                    break
                else:
                    sub = "".join(terms[term_idx:term_idx + str_len])
                    if province_flag:
                        if self.tries_lower['province'].search(sub):
                            provinces.append(sub)
                        else:
                            if tries['province'].startsWith(sub) == []:
                                province_flag = False
                    
                    if district_flag:
                        if self.tries_lower['district'].search(sub):
                            districts.append(sub)
                        else:
                            if tries['district'].startsWith(sub) == []:
                                district_flag = False
                    if ward_flag:
                        if self.tries_lower['ward'].search(sub):
                            wards.append(sub)
                        else:
                            if tries['ward'].startsWith(sub) == []:
                                ward_flag = False

        # pick a string in list

        # # sol1: pick the longest string
        # if provinces != []: 
        #     province = find_longest_str(provinces)
        # else:
        #     province = ""

        # if districts != []:
        #     district = find_longest_str(districts)
        # else:
        #     district = ""

        # if wards != []:
        #     ward = find_longest_str(wards)
        # else:
        #     ward = ""

        # sl2: pick by voting
        province, district, ward = pick_str_by_voting(provinces, districts, wards)


        # turn lower, wo space to original
        province = self.lower_to_org_dict.get_original_province(province)
        district = self.lower_to_org_dict.get_original_district(district)
        ward = self.lower_to_org_dict.get_original_ward(ward)

        print(province, district, ward)

        return {
            "province": province,
            "district": district,
            "ward": ward,
        }
    

## Marking (do not change from here)

In [122]:
# NOTE: DO NOT change this cell
# This cell is for downloading private test
!rm -rf test.json
# this link is public test
!gdown --fuzzy https://drive.google.com/file/d/1PBt3U9I3EH885CDhcXspebyKI5Vw6uLB/view?usp=sharing -O test.json

'rm' is not recognized as an internal or external command,
operable program or batch file.
'gdown' is not recognized as an internal or external command,
operable program or batch file.


In [123]:
# NOTE: DO NOT change this cell
# This cell is for scoring

TEAM_NAME = 'DEFAULT_NAME'  # This should be your team name
EXCEL_FILE = f'{TEAM_NAME}.xlsx'

import json
import time
with open('test.json', encoding='utf-8') as f:
    data = json.load(f)

summary_only = True
df = []
solution = Solution()
timer = []
correct = 0
for test_idx, data_point in enumerate(data):
    address = data_point["text"]

    ok = 0
    try:
        start = time.perf_counter_ns()
        result = solution.process(address)
        answer = data_point["result"]
        finish = time.perf_counter_ns()
        timer.append(finish - start)
        ok += int(answer["province"] == result["province"])
        ok += int(answer["district"] == result["district"])
        ok += int(answer["ward"] == result["ward"])
        df.append([
            test_idx,
            address,
            answer["province"],
            result["province"],
            int(answer["province"] == result["province"]),
            answer["district"],
            result["district"],
            int(answer["district"] == result["district"]),
            answer["ward"],
            result["ward"],
            int(answer["ward"] == result["ward"]),
            ok,
            timer[-1] / 1_000_000_000,
        ])
    except Exception as e:
        df.append([
            test_idx,
            address,
            answer["province"],
            "EXCEPTION",
            0,
            answer["district"],
            "EXCEPTION",
            0,
            answer["ward"],
            "EXCEPTION",
            0,
            0,
            0,
        ])
        pass
    correct += ok


    if not summary_only:
        # responsive stuff
        print(f"Test {test_idx:5d}/{len(data):5d}")
        print(f"Correct: {ok}/3")
        print(f"Time Executed: {timer[-1] / 1_000_000_000:.4f}")


print(f"-"*30)
total = len(data) * 3
score_scale_10 = round(correct / total * 10, 2)
if len(timer) == 0:
    timer = [0]
max_time_sec = round(max(timer) / 1_000_000_000, 4)
avg_time_sec = round((sum(timer) / len(timer)) / 1_000_000_000, 4)

import pandas as pd

df2 = pd.DataFrame(
    [[correct, total, score_scale_10, max_time_sec, avg_time_sec]],
    columns=['correct', 'total', 'score / 10', 'max_time_sec', 'avg_time_sec',],
)

columns = [
    'ID',
    'text',
    'province',
    'province_student',
    'province_correct',
    'district',
    'district_student',
    'district_correct',
    'ward',
    'ward_student',
    'ward_correct',
    'total_correct',
    'time_sec',
]

df = pd.DataFrame(df)
df.columns = columns

print(f'{TEAM_NAME = }')
print(f'{EXCEL_FILE = }')
print(df2)

%pip install xlsxwriter
writer = pd.ExcelWriter(EXCEL_FILE, engine='xlsxwriter')
df2.to_excel(writer, index=False, sheet_name='summary')
df.to_excel(writer, index=False, sheet_name='details')
writer.close()


['tuyenquang'] ['yenson', 'tuyenquang'] ['tanbinh', 'yenson']
2
['tuyenquang'] ['yenson', 'tuyenquang'] ['tanbinh', 'yenson']
[3] [2, 1] [0, 2]
Tuyên Quang Yên Sơn Yên Sơn
[] [] []
[] [] []
[] [] []
  
[] ['mytho'] []
[] ['mytho'] []
[] [0] []
 Mỹ Tho 
['tuyenquang'] ['phubinh', 'chiemhoa', 'tuyenquang'] ['phubinh']
3
['tuyenquang'] ['phubinh', 'chiemhoa', 'tuyenquang'] ['phubinh']
[3] [0, 2, 1] [2]
Tuyên Quang Chiêm Hóa Phú Bình
['hochiminh'] ['12', '1'] ['12', 'dakao']
2
['hochiminh'] ['12', '1'] ['12', 'dakao']
[4] [1, 2] [1, 2]
Hồ Chí Minh 1 Đa Kao
['hochiminh'] ['hocmon'] ['trungchanh', 'trungchanh']
1
['hochiminh'] ['hocmon'] ['trungchanh', 'trungchanh']
[3] [3] [2, 2]
Hồ Chí Minh Hóc Môn Trung Chánh
['quangninh'] ['campha'] ['cambinh']
1
['quangninh'] ['campha'] ['cambinh']
[2] [2] [2]
Quảng Ninh Cẩm Phả Cam Bình
['hungyen'] ['yenmy'] ['thanhlong']
1
['hungyen'] ['yenmy'] ['thanhlong']
[2] [2] [2]
Hưng Yên Yên Mỹ Thanh Long
['cantho'] ['vinhthanh'] ['thanhloi']
1
['cantho'] ['vi


[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip
